# Notebook para tratamento dos dados

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#style.use('seaborn')
%matplotlib inline


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from sklearn import preprocessing, metrics
# !pip install -U scikit-fuzzy
import sys
import skfuzzy as fuzz
sys.setrecursionlimit(1000)

In [4]:
data_column_names = ["Date", "Time", "Epoch", "Moteid", "Temperature", "Humidity", "Light", "Voltage"]
dplot = pd.read_csv("../data.txt", sep = ' ', names = data_column_names)

S13df = dplot[dplot['Moteid']==13]
S18df = dplot[dplot['Moteid']==18]

In [5]:
S14df = pd.read_csv('dfImputation.csv')
S14df.shape,S13df.shape,S18df.shape

((65571, 8), (31505, 8), (35312, 8))

In [6]:
S14df['3'] = 14.0
S14df.head()

,0,1,2,3,4,5,6,7
0,2004-03-31,03:38:15.757551,0,14.0,0.0000,-3.91901,11.04,2.03397
1,2004-03-21,19:07:14.467243,1,14.0,21.2722,48.15680,93.84,2.38522
2,2004-02-28,00:59:16.02785,2,14.0,0.0000,37.09330,45.08,2.69964
3,2004-02-28,01:03:16.33393,3,14.0,0.0000,38.46290,45.08,2.68742
4,2004-02-28,01:06:16.013453,4,14.0,0.0000,38.80390,45.08,2.68742


In [7]:
outliersIL = []
outliersILValues = []
for i, row in S13df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL.append(i)
        outliersILValues.append(row['Temperature'])
print(len(outliersIL))


6390


In [8]:
# Removendo outliers no Sensor
passo3= S13df.copy()
print(passo3.shape)
passo3= passo3.drop(outliersIL)
print(passo3.shape)
passo3.to_csv('S13FINAL.csv')

(31505, 8)
(25115, 8)


In [9]:
outliersIL2 = []
outliersILV2 = []
for i, row in S18df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL2.append(i)
        outliersILV2.append(row['Temperature'])
print(len(outliersIL2))


11649


In [10]:
# Removendo outliers no S2
passo3_3= S18df.copy()
print(passo3_3.shape)
passo3_3= passo3_3.drop(outliersIL2)
print(passo3_3.shape)
passo3_3.to_csv('S18FINAL.csv')

(35312, 8)
(23663, 8)


In [11]:
outliersIL_3 = []
outliersILV_3 = []
for i, row in S14df.iterrows():
    if row['4'] >=26.2:
        outliersIL_3.append(i)
        outliersILV_3.append(row['4'])
print(len(outliersIL_3))


0


In [12]:
# Removendo outliers no sensor
passo3_2= S14df.copy()
print(passo3_2.shape)
passo3_2= passo3_2.drop(outliersIL_3)
print(passo3_2.shape)
passo3_2.to_csv('S14FINAL.csv')

(65571, 8)
(65571, 8)


In [13]:
S14df_ = pd.read_csv('S14FINAL.csv')
S13df_ = pd.read_csv('S13FINAL.csv')
S18df_ = pd.read_csv('S18FINAL.csv')
S14df_['3'] = 14
S14df_.shape, S13df_.shape, S18df_.shape

((65571, 9), (25115, 9), (23663, 9))

In [14]:
S14df_.head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,2004-03-31,03:38:15.757551,0,14,0.0000,-3.91901,11.04,2.03397
1,1,2004-03-21,19:07:14.467243,1,14,21.2722,48.15680,93.84,2.38522
2,2,2004-02-28,00:59:16.02785,2,14,0.0000,37.09330,45.08,2.69964
3,3,2004-02-28,01:03:16.33393,3,14,0.0000,38.46290,45.08,2.68742
4,4,2004-02-28,01:06:16.013453,4,14,0.0000,38.80390,45.08,2.68742


# Sistema Fuzzy

In [15]:
# Modelo Fuzzy
temp = np.arange(-26.2, 26.2, 0.1)
tempF = np.arange(-26.2, 26.2, 0.1)

# Generate fuzzy membership functions
temp_lo = fuzz.trimf(temp, [-26.2, 0.0, 5.0])  
temp_md = fuzz.trimf(temp, [0.0, 5.0, 20.0])      
temp_hi = fuzz.trimf(temp, [17.20, 22.8, 26.2]) 


mfx = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2])


tempFinal_lo = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2]) 
tempFinal_md = fuzz.trimf(tempF, [0, 5.0, 20.0]) 
tempFinal_hi = fuzz.trimf(tempF, [17.20, 22.8, 26.2])


In [20]:
def fuzification(i, data, data1, data2): 
    if data['3'].ix[i]== 14:
        temp_level_lo0 = fuzz.interp_membership(temp, temp_lo,  data1['Temperature'][i]) # ex: [52, [51, 52]]
        temp_level_md0 = fuzz.interp_membership(temp, temp_md,  data1['Temperature'][i])
        temp_level_hi0 = fuzz.interp_membership(temp, temp_hi,  data1['Temperature'][i])

        temp_level_lo1 = fuzz.interp_membership(temp, temp_lo,  data2['Temperature'][i])
        temp_level_md1 = fuzz.interp_membership(temp, temp_md,  data2['Temperature'][i])
        temp_level_hi1 = fuzz.interp_membership(temp, temp_hi,  data2['Temperature'][i])

        active_rule1_0 = np.fmin(temp_level_lo0, temp_level_lo1)
        active_rule1_1 = np.fmin(temp_level_lo0, temp_level_md1)
        active_rule1_2 = np.fmin(temp_level_lo0, temp_level_hi1)
        active_rule1_3 = np.fmin(temp_level_md0, temp_level_lo1)
        active_rule1_4 = np.fmin(temp_level_md0, temp_level_md1)
        active_rule1_5 = np.fmin(temp_level_md0, temp_level_hi1)
        active_rule1_6 = np.fmin(temp_level_hi0, temp_level_lo1)
        active_rule1_7 = np.fmin(temp_level_hi0, temp_level_md1)
        active_rule1_8 = np.fmin(temp_level_hi0, temp_level_hi1)


        active_rule1_Final_0 = np.fmax(active_rule1_0, tempFinal_lo)
        active_rule1_Final_1 = np.fmax(active_rule1_1, tempFinal_lo)
        active_rule1_Final_2 = np.fmax(active_rule1_2, tempFinal_md)
        active_rule1_Final_3 = np.fmax(active_rule1_3, tempFinal_lo)
        active_rule1_Final_4 = np.fmax(active_rule1_4, tempFinal_hi)
        active_rule1_Final_5 = np.fmax(active_rule1_5, tempFinal_hi)
        active_rule1_Final_6 = np.fmax(active_rule1_6, tempFinal_md)
        active_rule1_Final_7 = np.fmax(active_rule1_7, tempFinal_hi)
        active_rule1_Final_8 = np.fmax(active_rule1_8, tempFinal_hi)

        r1_0 = np.fmax(active_rule1_Final_8, np.fmax(active_rule1_Final_7, np.fmax(active_rule1_Final_6, active_rule1_Final_5) ) )  
        r1_1 = np.fmax(active_rule1_Final_5, np.fmax(active_rule1_Final_3, np.fmax(active_rule1_Final_2, np.fmax(active_rule1_Final_1, active_rule1_Final_0)) ) )  
        aggregated = np.fmax(r1_0,r1_1)
        return aggregated  


In [17]:
def defuzz(x, mfx, mode):
    """
    Defuzzification of a membership function, returning a defuzzified value
    of the function at x, using various defuzzification methods.

    Parameters
    ----------
    x : 1d array or iterable, length N
        Independent variable.
    mfx : 1d array of iterable, length N
        Fuzzy membership function.
    mode : string
        Controls which defuzzification method will be used.
        * 'centroid': Centroid of area
        * 'bisector': bisector of area
        * 'mom'        : mean of maximum
        * 'som'        : min of maximum
        * 'lom'        : max of maximum

    Returns
    -------
    u : float or int
        Defuzzified result.

    See also
    --------
    CENTROID, DCENTROID

    """
    mode = mode.lower()
    x = x.ravel()
    mfx = mfx.ravel()
    n = len(x)
    assert n == len(mfx), 'Length of x and fuzzy membership function must be \
                          identical.'

    if 'Centroid' in mode or 'bisector' in mode:
        tot_area = mfx.sum()
        assert tot_area != 0, 'Total area is zero in defuzzification!'

        if 'Centroid' in mode:
            return centroid(x, mfx)

        elif 'bisector' in mode:
            tmp = 0
            for k in range(n):
                tmp += mfx[k]
                if tmp >= tot_area / 2.:
                    return x[k]

    elif 'mom' in mode:
        return np.mean(x[mfx == mfx.max()])

    elif 'som' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).min()][0]

    elif 'lom' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).max()][0]

    else:
        raise ValueError('The input for `mode`, %s, was incorrect.' % (mode))



In [21]:
aux = 0.0
for i, row in S14df_.iterrows():
        if S14df_['4'][i] == 0.0:            
            try:
                S14df_.at[i, '4'] = defuzz(tempF,fuzification(i, S14df_,  S13df_,S18df_ ), 'lom')
                aux = defuzz(tempF,fuzification(i, S14df_, S13df_,S18df_ ), 'lom')
            except:
                S14df_.at[i, '4'] = aux
                
S14df_.tail()

,Unnamed: 0,0,1,2,3,4,5,6,7
65566,65566,2004-03-18,21:41:55.763562,65531,14,17.1000,43.4191,45.08,2.43410
65567,65567,2004-03-21,19:04:31.936814,65532,14,21.3212,48.1568,93.84,2.38522
65568,65568,2004-03-18,21:42:26.695299,65533,14,17.1000,43.2525,45.08,2.43410
65569,65569,2004-03-18,21:42:48.230907,65534,14,17.1000,43.2525,45.08,2.43410
65570,65570,2004-03-21,19:06:00.392624,65535,14,21.2722,48.1568,93.84,2.38522


In [19]:
print(S14df_.shape)
S14df_.to_csv('finalsensor.csv', index=False)


(65571, 9)


# Final